In [1]:
import pandas as pd
# {"headers": {"Weather": ["id", "recordDate", "temperature"]}, 
# "rows": {"Weather": [[1, "2015-01-01", 10], [2, "2015-01-02", 25], [3, "2015-01-03", 20], [4, "2015-01-04", 30]]}}
pd.DataFrame([[1, "2015-01-01", 10], [2, "2015-01-02", 25], [3, "2015-01-03", 20], [4, "2015-01-04", 30]], columns=["id", "recordDate", "temperature"]).to_csv("./Weather.txt", index=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("rising temperature").config("pyspark.sql.shuffle.partition", "4").getOrCreate()

In [3]:
w = spark.read.option("header", True).csv("./Weather.txt")
w.show()

+---+----------+-----------+
| id|recordDate|temperature|
+---+----------+-----------+
|  1|2015-01-01|         10|
|  2|2015-01-02|         25|
|  3|2015-01-03|         20|
|  4|2015-01-04|         30|
+---+----------+-----------+



Write an SQL query to find all dates' Id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.

The query result format is in the following example.

In [16]:
w.printSchema()
w=w.withColumn("recordDate", w.recordDate.cast("date"))
w.printSchema()

root
 |-- id: string (nullable = true)
 |-- recordDate: string (nullable = true)
 |-- temperature: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- recordDate: date (nullable = true)
 |-- temperature: string (nullable = true)



In [29]:

def changeColumnName(w, n):
    for i in w.columns:
        w = w.withColumnRenamed(i, i+f"_{n}")
    return w
w1 = changeColumnName(w, 1)
w1.show()

+----+------------+-------------+
|id_1|recordDate_1|temperature_1|
+----+------------+-------------+
|   1|  2015-01-01|           10|
|   2|  2015-01-02|           25|
|   3|  2015-01-03|           20|
|   4|  2015-01-04|           30|
+----+------------+-------------+



In [30]:
from pyspark.sql.functions import datediff

# w1 = w.withColumnRenamed("recordDate", "rd").withColumnRenamed("temperature","t").withColumnRenamed("id", "i")
w.join(w1).where(datediff(w.recordDate, w1.recordDate_1)==1).where(w1.temperature_1<w.temperature ).select("id").show()

+---+
| id|
+---+
|  2|
|  4|
+---+

